In [1]:
# -*- coding: utf-8 -*-
from __future__ import unicode_literals
# text in Western (Windows 1252)

import pickle
import numpy as np
# import StringIO
import math
from keras import optimizers, metrics
from keras.models import Model
from keras.layers import Dense, Dropout, Input
from keras.layers.merge import concatenate
from keras import regularizers
from keras.layers.convolutional import Conv1D
from keras.layers.convolutional import MaxPooling1D
from keras.constraints import maxnorm
from keras.layers import Flatten
from keras.optimizers import SGD
from keras.models import load_model
# from keras import backend as Input
np.random.seed(7)

Using Theano backend.


In [2]:
# %run ../../../prepare_data.py

import sys
sys.path.insert(0, '../../../')
from prepare_data import *

In [7]:
# %run ../../../prepare_data.py
# X_train, X_other_features_train, y_train, X_test, X_other_features_test, y_test, X_validate, X_other_features_validate, y_validate = generate_syllable_inputs('../../internal_representations/inputs/content_shuffle_vector.h5', '../../internal_representations/inputs/shuffle_vector')
# save_inputs('../../internal_representations/inputs/shuffeled_syllable_train_inputs.h5', X_train, y_train, other_features = X_other_features_train)
# save_inputs('../../internal_representations/inputs/shuffeled_syllable_test_inputs.h5', X_test, y_test, other_features = X_other_features_test)
# save_inputs('../../internal_representations/inputs/shuffeled_syllable_validate_inputs.h5', X_validate, y_validate,  other_features = X_other_features_validate)
# X_train, X_other_features_train, y_train = load_inputs('../../internal_representations/inputs/shuffeled_syllable_train_inputs.h5', other_features=True)
# X_test, X_other_features_test, y_test = load_inputs('../../internal_representations/inputs/shuffeled_syllable_test_inputs.h5', other_features=True)
# X_validate, X_other_features_validate, y_validate = load_inputs('../../internal_representations/inputs/shuffeled_syllable_validate_inputs.h5', other_features=True)

# syllable_letters_translator = create_syllable_letters_translator(max_syllable, syllable_dictionary, dictionary, vowels)

data = Data('sl')
data.generate_data('shuffled_syllable_reversed_train',
                   'shuffled_syllable_reversed_test',
                   'shuffled_syllable_reversed_validate')

LOADING DATA...
LOAD SUCCESSFUL!


In [8]:
%run ../../../prepare_data.py
gen = data.generator('train', 16)
test = next(gen)

In [21]:
# prints test word from generator
# print(X_train[0])
# print(syllable_dictionary[test[0][0][0][0].tolist().index(1)])
print(np.array(test[0][0]).shape)
for el in test[0][0][0]:
    syllable = ''
    for pos in range(0, len(el), 36):
        syllable += data.decode_x([el[pos:pos+36]], dictionary)
    print(syllable)
#     print(el)
    
#     print(syllable_dictionary[el.tolist().index(1)])

(16, 11, 252)
na
ja
ktu
a
lnej
ša







In [25]:
num_examples = len(data.x_train) # training set size

nn_output_dim = 10
nn_hdim = 516
batch_size = 16
actual_epoch = 1
num_fake_epoch = 20



conv_input_shape=(10, 252)
othr_input = (140, )

conv_input = Input(shape=conv_input_shape, name='conv_input')
# x_conv = Conv1D(43, (3), padding='same', activation='relu')(conv_input)
x_conv = Conv1D(44, (3), padding='same', activation='relu')(conv_input)
x_conv = MaxPooling1D(pool_size=2)(x_conv)
x_conv = Flatten()(x_conv)
# x_conv = Dense(516, activation='relu', kernel_constraint=maxnorm(3))(x_conv)

othr_input = Input(shape=othr_input, name='othr_input')
# x_othr = Dense(256, input_dim=167, activation='relu')(othr_input)
# x_othr = Dropout(0.3)(x_othr)
# x_othr = Dense(512, activation='relu')(othr_input)
# x_othr = Dropout(0.3)(x_othr)
# x_othr = Dense(256, activation='relu')(othr_input)

x = concatenate([x_conv, othr_input])
# x = Dense(1024, input_dim=(516 + 256), activation='relu')(x)
x = Dense(256, activation='relu')(x)
x = Dropout(0.3)(x)
x = Dense(512, activation='relu')(x)
x = Dropout(0.3)(x)
x = Dense(256, activation='relu')(x)
x = Dropout(0.2)(x)
x = Dense(nn_output_dim, activation='sigmoid')(x)




model = Model(inputs=[conv_input, othr_input], outputs=x)
opt = optimizers.Adam(lr=1E-4, beta_1=0.9, beta_2=0.999, epsilon=1e-08)
model.compile(loss='binary_crossentropy', optimizer=opt, metrics=[actual_accuracy,])

In [28]:
history = model.fit_generator(data.generator('train', batch_size), 
                              data.x_train.shape[0]/(batch_size * num_fake_epoch), 
                              epochs=actual_epoch*num_fake_epoch, 
                              validation_data=data.generator('test', batch_size), 
                              validation_steps=data.x_test.shape[0]/(batch_size * num_fake_epoch))

Epoch 1/20
1345/1344 [==============================] - 25s - loss: 0.1859 - actual_accuracy: 0.3887 - val_loss: 0.1213 - val_actual_accuracy: 0.6196
Epoch 2/20
1345/1344 [==============================] - 27s - loss: 0.1119 - actual_accuracy: 0.6344 - val_loss: 0.1016 - val_actual_accuracy: 0.7090
Epoch 3/20
1345/1344 [==============================] - 27s - loss: 0.0980 - actual_accuracy: 0.7034 - val_loss: 0.0947 - val_actual_accuracy: 0.7462
Epoch 4/20
1345/1344 [==============================] - 28s - loss: 0.0913 - actual_accuracy: 0.7341 - val_loss: 0.0880 - val_actual_accuracy: 0.7577
Epoch 5/20
1345/1344 [==============================] - 28s - loss: 0.0854 - actual_accuracy: 0.7572 - val_loss: 0.0885 - val_actual_accuracy: 0.7638
Epoch 6/20
1345/1344 [==============================] - 29s - loss: 0.0795 - actual_accuracy: 0.7782 - val_loss: 0.0820 - val_actual_accuracy: 0.7883
Epoch 7/20
1345/1344 [==============================] - 30s - loss: 0.0755 - actual_accuracy: 0.7887

In [27]:
name = '1_epoch'
model.save(name + '.h5')
output = open(name + '_history.pkl', 'wb')
pickle.dump(history.history, output)
output.close()

In [15]:
# dictionary, max_word, max_num_vowels, content, vowels, accetuated_vowels = create_dict()
# train_content, test_content, validate_content = split_content(content, 0.2, '../../internal_representations/inputs/content_shuffle_vector.h5')
# feature_dictionary = create_feature_dictionary()
# syllable_dictionary = create_syllables_dictionary(content, vowels)
# max_syllable = get_max_syllable(syllable_dictionary)


content = data._read_content('../../../data/SlovarIJS_BESEDE_utf8.lex')
dictionary, max_word, max_num_vowels, vowels, accented_vowels = data._create_dict(content)
feature_dictionary = data._create_feature_dictionary()
syllable_dictionary = data._create_syllables_dictionary(content, vowels)

In [8]:
%run ../../../prepare_data.py
# generate_X_and_y(dictionary, max_word, max_num_vowels, content, vowels, accetuated_vowels, feature_dictionary)